In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import cufflinks as cf
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import norm
from scipy.optimize import fsolve
from pylab import mpl
from mpl_toolkits.mplot3d import Axes3D

In [2]:
np.random.seed(1853)
mpl.rcParams["font.sans-serif"]=["SimHei"]
mpl.rcParams["axes.unicode_minus"]=False
cf.go_offline()

In [3]:
color = ["rgb(220,38,36)","rgb(43,71,80)","rgb(69,160,162)",
"rgb(232,122,89)","rgb(125,202,169)","rgb(100,158,125)",
"rgb(220,128,24)","rgb(200,159,145)","rgb(108,109,108)",
"rgb(79,98,104)","rgb(199,204,207)"]

In [4]:
(S0, r, q, T, sigma) = (1, 0.02, 0.01, 1, 0.5)
(Nsim, Nt) = (10, 1000)
t = np.linspace(0,T,Nt)
dt = np.diff(t)

z = np.random.randn(Nsim, Nt-1)
A = (r-q-0.5*(sigma)**2)*dt + sigma*(z*np.sqrt(dt))

lnS0 = np.tile(np.log(S0),(Nsim,1))
lnS = lnS0 + np.cumsum(A, axis=1)
lnS = np.hstack( (lnS0, lnS) )
S = np.exp(lnS)

In [5]:
label = [ x + ' ' + str(y) for x, y in zip(['路径']*Nsim, np.arange(1,Nsim+1)) ]
df = pd.DataFrame(S.T, index=t, columns=label)
df.iplot( xTitle='t', 
          yTitle='S', 
          title='Black-Scholes Model下的资产价格路径模拟', 
          theme='ggplot' )

In [6]:
def blackscholes( S0=100,K=100,r=0.01,q=0.01,T=1,sigma=0.2,omega=1):
    discount = np.exp(-r*T)
    forward = S0*np.exp((r-q)*T)
    moneyness = np.log(forward/K)
    vol_sqrt_T = sigma*np.sqrt(T)

    d1 = moneyness / vol_sqrt_T
    d2 = d1 - vol_sqrt_T

    V = omega * discount * (forward*norm.cdf(omega*d1) - K*norm.cdf(omega*d2))
    return V

In [7]:
(S0, K, r, q, T, sigma, omega) = (100, 95, 0.01, 0, 1, 0.1, 1)
V_BS = blackscholes(S0, K, r, q, T, sigma, omega)
V_BS

7.541787696964167

In [8]:
def get_BS_IV(price,S0,K,r,q,T,omega):
    x0 = 0.2
    obj = lambda x: blackscholes(S0,K,r,q,T,x,omega)-price
    x = fsolve(obj,x0)
    return x[0]

In [9]:
get_BS_IV(V_BS,S0,K,r,q,T,omega)

0.10000000000000031

In [10]:
#Bachelier Model

In [11]:
(S0, r, q, T, sigma) = (1, 0.02, 0.01, 10, 0.5)
(Nsim, Nt) = (10, 1000)
t = np.linspace(0,T,Nt)
dt = np.diff(t)
z = np.random.randn(Nsim, Nt-1)

A = np.exp((r-q)*t)

if r == q:
    B = sigma*(z*np.sqrt(dt)) / A[1:]
else:
    B = sigma*(z*np.sqrt( (np.exp(2*(r-q)*dt)-1) / (2*(r-q)) )) / A[1:]

discS0 = np.tile( S0/A[0], (Nsim,1) )
discS = discS0 + np.cumsum(B, axis=1)
discS = np.hstack( (discS0, discS) )
S = discS * A

In [19]:
label = [ x + ' ' + str(y) for x, y in zip(['Path']*Nsim, np.arange(1,Nsim+1)) ]
df = pd.DataFrame(S.T, index=t, columns=label)
df.iplot( xTitle='t', 
          yTitle='S', 
          title='Simulation of asset price path with Bachelier Model', 
          theme='ggplot' )

In [13]:
def bachelier(S0=100,K=100,r=0.01,q=0.01,T=1,sigma=0.2,omega=1):
    discount = np.exp(-r*T)
    forward = S0*np.exp((r-q)*T)
    moneyness = forward - K

    if r == q:
        vol_sqrt_T = sigma * np.sqrt(T)
    else:
        vol_sqrt_T = sigma*np.sqrt((np.exp(2*(r-q)*T)-1)/(2*(r-q)))

    d = moneyness /vol_sqrt_T

    V = discount *(omega*moneyness*norm.cdf(omega*d)+vol_sqrt_T*norm.pdf(d))
    return V

In [14]:
(S0, K, r, q, T, sigma, omega) = (100, 95, 0.01, 0, 1, 0.1, 1)
V_BL = bachelier(S0, K, r, q, T, sigma, omega)
V_BL

5.945265793829019

In [15]:
V_BL = bachelier(S0, K, r, q, T, sigma*S0, omega)
V_BL

7.630422479463851

In [16]:
(S0, K) = (-30, 30)
V_BL = bachelier(S0, K, r, q, T, sigma*np.abs(S0), omega)
V_BL

4.0827286280683055e-90

In [17]:
def get_BL_IV(price,S0,K,r,q,T,omega):
    x0=100
    obj = lambda x:bachelier(S0,K,r,q,T,x,omega) - price
    x = fsolve(obj,x0)
    return x[0]

In [18]:
(S0, K, r, q, T, omega) = (100, 95, 0.01, 0, 1, 1)
BS_sigma = 0.1
BS_price = blackscholes(S0, K, r, q, T, BS_sigma, omega)
BL_price = BS_price
BL_sigma = get_BL_IV( BL_price, S0, K, r, q, T, omega )
BL_sigma

9.731761991794372